In [86]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Binarizer
from sklearn.linear_model import SGDClassifier
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer

In [4]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [46]:
y = titanic['survived']
X = titanic.drop(['survived'], axis = 1)

### make_column_transformer
Les colonnes de notre X sont qualitatives et quantitatives. Il y aura donc des erreurs sur les transformers du pipeline<br>
Pour resoudre ce probleme on cree dans notre pipeline un mecanisme permet de donner certains colonne a certains transformers<br><br>
On utilise la fonction make_column_transformer

In [16]:
numerical_features = ['pclass', 'age', 'fare']
categorical_features = ['sex', 'deck', 'alone']

In [30]:
numerical_pipeline = make_pipeline(SimpleImputer(), StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy = 'most_frequent'), OneHotEncoder())

In [34]:
preprocessor = make_column_transformer((numerical_pipeline, numerical_features), (categorical_pipeline, categorical_features))

In [54]:
model = make_pipeline(preprocessor, SGDClassifier())

In [56]:
model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'deck', 'alone'])])),
                ('sgdclassifier', SGDClassifier())])

### make_column_selector
Permet d'indiquer le type de variable qu'on veut selectionner

In [69]:
numerical_features = make_column_selector(dtype_include = np.number)
categorical_features = make_column_selector(dtype_exclude = np.number)

# Et après on recompile notre code. On aura le meme resultat

In [75]:
numerical_pipeline = make_pipeline(SimpleImputer(), StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy = 'most_frequent'), OneHotEncoder())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features), (categorical_pipeline, categorical_features))

model = make_pipeline(preprocessor, SGDClassifier())

model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f20cf0d1c40>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f20cf0d2240>)])),
                ('sgdclassifier', SGDClassifier())])

### make_union
Pour creer des pipeline parallele

In [96]:
features =  X[['age', 'fare']].dropna(axis = 0)

In [100]:
pipeline = make_union(StandardScaler(), Binarizer())
pipeline.fit(features)

FeatureUnion(transformer_list=[('standardscaler', StandardScaler()),
                               ('binarizer', Binarizer())])

In [106]:
# On obtient 4 colonne, les 2 premieres sont celles qui nt été standardisé et les deux dernieres binarisé
pipeline.transform(features)

array([[-0.53037664, -0.51897787,  1.        ,  1.        ],
       [ 0.57183099,  0.69189675,  1.        ,  1.        ],
       [-0.25482473, -0.50621356,  1.        ,  1.        ],
       ...,
       [-0.73704057, -0.08877362,  1.        ,  1.        ],
       [-0.25482473, -0.08877362,  1.        ,  1.        ],
       [ 0.15850313, -0.50952283,  1.        ,  1.        ]])